In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feature_bagging import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import datetime
from sklearn.preprocessing import MinMaxScaler

# Carga de datos

In [2]:
df_tb_productos = pd.read_csv('../data/tb_productos_descripcion.txt', sep='\t')
df_sell_in = pd.read_csv('../data/sell-in.txt', sep='\t')
df_predecir = pd.read_csv('../data/productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv('../data/tb_stocks.txt', sep='\t')

In [3]:
#### MARCOS
df_tb_stocks['periodo'] = pd.to_datetime(df_tb_stocks['periodo'], format='%Y%m')
#df_sell_in = df_sell_in[df_sell_in['periodo']<201808]
df_sell_in['tn_2'] = df_sell_in['tn'].shift(-2) #OBTENGO TN 2 MESES ADELANTE
#df_sell_in['tn_2'] = np.where(df_sell_in['periodo'].isin([201805, 201804]), np.nan, df_sell_in['tn_2'])
#df_sell_in = df_sell_in[df_sell_in['periodo']<201806]

In [4]:
df_tb_stocks['product_id'] = df_tb_stocks['product_id'].astype(int)
df_tb_stocks['stock_final'] = df_tb_stocks['stock_final'].astype(float)
df_tb_productos['product_id'] = df_tb_productos['product_id'].astype(int)
df_tb_productos['sku_size'] = df_tb_productos['sku_size'].astype(int)
df_sell_in['periodo'] = pd.to_datetime(df_sell_in['periodo'], format='%Y%m')
df_sell_in['product_id'] = df_sell_in['product_id'].astype(int)
df_sell_in['customer_id'] = df_sell_in['customer_id'].astype(int)
df_sell_in['cust_request_qty'] = df_sell_in['cust_request_qty'].astype(int)
df_sell_in['cust_request_tn'] = df_sell_in['cust_request_tn'].astype(float)
df_sell_in['tn'] = df_sell_in['tn'].astype(float)
df_sell_in['plan_precios_cuidados'] = df_sell_in['plan_precios_cuidados'].astype(bool)
# Join tb_productos to sell_in on product_id
df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')
df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')
df_final["trimestre"] = df_final.periodo.dt.quarter
#all_periods = pd.DataFrame()
#all_periods['fecha'] = pd.date_range(start='2017-01-01', periods=36, freq='MS')
#df_final = pd.merge(all_periods, df_final, on=['fecha'], how='left')
df_final.set_index('fecha', inplace=True)
df_final.index = df_final.index.to_period('M')

In [5]:
df_final['diff_tn_tn2'] =  df_final['tn_2'] - df_final['tn'] #NUEVA CLASE

In [6]:
n_lags = 36
# Create lag variables for 'cust_request_qty', 'cust_request_tn', and 'tn' from lag -1 to -12
for lag in range(1, n_lags + 1):
    #df_final[f'cust_request_qty_lag_{lag}'] = df_final['cust_request_qty'].shift(lag)
    df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
    #df_final[f'stock_vendido_lag_{lag}'] = df_final['stock_vendido'].shift(lag)
    df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
    df_final[f'tn_lag_{lag}'] = df_final['tn'].shift(lag)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\2160874499.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'tn_lag_{lag}'] = df_final['tn'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\2160874499.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\2160874499.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

In [7]:
#DELTA DE LAGS 1 ANTERIOR

for lag in range(1, n_lags):
    df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
        df_final[f'cust_request_tn_lag_{lag+1}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
        df_final[f'stock_final_lag_{lag+1}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+1}'] = (
        df_final[f'tn_lag_{lag+1}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\1785296116.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\1785296116.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\1785296116.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [8]:
#Mes actual / (lag 2 + lag 3) 

# Calcular el ratio del valor actual con respecto a la suma de los dos lags anteriores
for lag in range(3, n_lags + 1):
    df_final[f'ratio_cust_request_tn_{lag}'] = (
        df_final[f'cust_request_tn_lag_{lag}'] / (
            df_final[f'cust_request_tn_lag_{lag-1}'] + df_final[f'cust_request_tn_lag_{lag-2}']
        )
    )
    df_final[f'ratio_stock_final_{lag}'] = (
        df_final[f'stock_final_lag_{lag}'] / (
            df_final[f'stock_final_lag_{lag-1}'] + df_final[f'stock_final_lag_{lag-2}']
        )
    )
    df_final[f'ratio_tn_{lag}'] = (
        df_final[f'tn_lag_{lag}'] / (
            df_final[f'tn_lag_{lag-1}'] + df_final[f'tn_lag_{lag-2}']
        )
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\2974401302.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_cust_request_tn_{lag}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\2974401302.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_stock_final_{lag}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\2974401302.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid

In [9]:
#Calcular ventas por trimestre
df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
#Calcular ventas por trimestre por cliente
df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\3111183930.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\3111183930.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')


In [10]:
#MEDIAS MOVILES
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\3749147370.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\3749147370.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Tem

In [11]:
#MEDIAS MOVILES POR CLIENTE
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_customer_{window}'] = df_final.groupby(['product_id','customer_id'])['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\3819369804.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_customer_{window}'] = df_final.groupby(['product_id','customer_id'])['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\3819369804.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_customer_{window}'] = df_final.groupby(['product_id','customer_id'])['tn'].transform(lambda x: x.rolling(window, min_p

In [12]:
#Variables Dummies si es el max o el min de cierta cantidad de meses
months = [3, 6, 9, 12]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for i in months:
    df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
    df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())

    # Crear las dummies
    df_final[f'dummy_max_{i}m'] = np.where(df_final['tn'] == df_final[f'max_{i}m'], 1, 0)
    df_final[f'dummy_min_{i}m'] = np.where(df_final['tn'] == df_final[f'min_{i}m'], 1, 0)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\525506276.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\525506276.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\525506276.py:10: Performance

In [13]:
df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')

C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\4104376824.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')


In [14]:
df_final.reset_index(inplace=True)
df_final['month'] = df_final['periodo'].dt.month
df_final['year'] = df_final['periodo'].dt.year

df_final['plan_precios_cuidados'] =df_final['plan_precios_cuidados'].astype(bool)
#df_final['dias_fin_trimestre'] = df_final['dias_fin_trimestre'].dt.days.astype(int)
df_final["trimestre"] = df_final.periodo.dt.quarter

#df_final = df_final.drop(columns=['periodo'])
df_final = df_final.drop(columns=['fecha'])
df_final['periodo']
df_final.set_index('periodo', inplace=True)
df_final.index = df_final.index.to_period('M')


C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\1832065075.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final.reset_index(inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\1832065075.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['month'] = df_final['periodo'].dt.month
C:\Users\Admin\AppData\Local\Temp\ipykernel_14800\1832065075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [15]:
df_parquet = df_final
""" df_parquet["cat1"] = df_parquet["cat1"].astype("str")
df_parquet["cat2"] = df_parquet["cat2"].astype("str")
df_parquet["cat3"] = df_parquet["cat3"].astype("str")   
df_parquet["brand"] = df_parquet["brand"].astype("str") """
df_final["cat1"] = df_final["cat1"].astype("category")
df_final["cat2"] = df_final["cat2"].astype("category")
df_final["cat3"] = df_final["cat3"].astype("category")
df_final["brand"] = df_final["brand"].astype("category")
df_final["descripcion"] = df_final["descripcion"].astype("category")
df_parquet.to_parquet('./FE_dataset2018.parquet', engine='pyarrow')  